In [159]:
import os 
import pandas as pd


SCORE_DIR = "data/scores"

In [161]:
box_scores = os.listdir(SCORE_DIR)
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html")]

In [173]:
from bs4 import BeautifulSoup

def parse_html(box_score):
    with open(box_score) as f:
        html = f.read()

    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    return soup

In [175]:
def read_season_info(soup):
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all('a')]
    season = os.path.basename(hrefs[1]).split("_")[0]
    return season

In [177]:
def read_line_score(soup):
    line_score = pd.read_html(str(soup), attrs={"id" : "line_score"})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols

    line_score = line_score[["team", "total"]]
    return line_score

In [180]:
def read_stats(soup, team, stat):
    df = pd.read_html(str(soup), attrs = {'id': f'box-{team}-game-{stat}'}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df

In [ ]:
games = []
base_cols = None
for box_score in box_scores:
    soup = parse_html(box_score)

    line_score = read_line_score(soup)
    teams = list(line_score["team"])

    summaries = []
    for team in teams:
        basic = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")

        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        totals.index = totals.index.str.lower()

        maxes = pd.concat([basic.iloc[:-1].max(), advanced.iloc[:-1].max()])
        maxes.index = maxes.index.str.lower() + "_max"

        summary = pd.concat([totals, maxes])
        
        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep="first"))
            base_cols = [b for b in base_cols if "bpm" not in b]
        
        summary = summary[base_cols]
        
        summaries.append(summary)
    summary = pd.concat(summaries, axis=1).T

    game = pd.concat([summary, line_score], axis=1)

    game["home"] = [0,1]

    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"

    full_game = pd.concat([game, game_opp], axis=1)
    full_game["season"] = read_season_info(soup)
    
    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format="%Y%m%d")
    
    full_game["won"] = full_game["total"] > full_game["total_opp"]
    games.append(full_game)
    
    if len(games) % 100 == 0:
        print(f"{len(games)} / {len(box_scores)}")

100 / 9255
200 / 9255
300 / 9255
400 / 9255
500 / 9255
600 / 9255
700 / 9255
800 / 9255
900 / 9255
1000 / 9255
1100 / 9255
1200 / 9255
1300 / 9255
1400 / 9255
1500 / 9255
1600 / 9255


In [187]:
games_df = pd.concat(games, ignore_index=True)

In [188]:
games_df

,mp,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,240.0,240.0,38.0,72.0,0.528,16.0,37.0,0.432,18.0,21.0,...,26.2,26.8,155.0,123.0,MIA,107,1,2023,2022-11-07,True
1,240.0,240.0,40.0,84.0,0.476,14.0,39.0,0.359,13.0,15.0,...,41.0,37.3,160.0,121.0,POR,110,0,2023,2022-11-07,False
2,240.0,240.0,41.0,78.0,0.526,8.0,24.0,0.333,15.0,19.0,...,28.6,41.1,250.0,125.0,DAL,90,1,2023,2022-12-14,True
3,240.0,240.0,29.0,74.0,0.392,13.0,38.0,0.342,19.0,26.0,...,12.6,33.0,183.0,110.0,CLE,105,0,2023,2022-12-14,False
4,240.0,240.0,37.0,87.0,0.425,7.0,33.0,0.212,32.0,35.0,...,20.0,32.3,226.0,117.0,TOR,126,1,2023,2022-12-07,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14939,240.0,240.0,37.0,88.0,0.420,11.0,29.0,0.379,24.0,27.0,...,25.8,31.0,185.0,126.0,NYK,103,0,2022,2021-11-21,True
14940,240.0,240.0,46.0,102.0,0.451,7.0,35.0,0.200,20.0,27.0,...,37.5,32.8,205.0,120.0,LAL,125,1,2019,2019-02-27,False
14941,240.0,240.0,49.0,94.0,0.521,8.0,25.0,0.320,19.0,31.0,...,69.4,82.7,155.0,122.0,NOP,119,0,2019,2019-02-27,True
14942,240.0,240.0,42.0,90.0,0.467,11.0,30.0,0.367,14.0,17.0,...,50.0,34.5,163.0,122.0,WAS,105,1,2022,2022-01-21,True


In [191]:
games_df.to_csv("nba_games.csv")